# Домашняя работа №2. Работа с POS-тэггерами

_Екатерина Волошина_

Импорт нужных модулей:

In [255]:
from sklearn.metrics import accuracy_score

In [256]:
import nltk
import re
import spacy
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from flair.data import Sentence
from flair.models import SequenceTagger
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

morph_parse = MorphAnalyzer()
m = Mystem()

In [257]:
from nltk.tokenize import WordPunctTokenizer

wpt = WordPunctTokenizer()

In [258]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem


--2020-10-16 19:17:11--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.242, 5.45.205.245, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.242|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskstoredata10.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-16 19:17:11--  http://cache-mskstoredata10.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskstoredata10.cdn.yandex.net (cache-mskstoredata10.cdn.yandex.net)... 37.9.96.21, 2a02:6b8:0:3706::19
Connecting to cache-mskstoredata10.cdn.yandex.net (cache-mskstoredata10.cdn.yandex.net)|37.9.96.21|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.3’

mystem-

## Тексты

В тексте на русском сложными для pos-теггинга могут быть глагол "печь", так как pos-тэггер может спутать с существительным, так же может спутать глагол "пила" с существительным. Вероятно, возникнут проблемы с "родными" и "близкими", которые здесь употреблены как существительные, а pos-тэггер может определить как прилагательные. Возможно, будут сложными местоимения-прилагательные, которые в правильных ответах отмечаются как местоимения, а pos-тэггер может отметить как прилагательные

In [259]:
russian_text = """Мое любимое хобби – печь пироги. 
Я стараюсь правильно питаться, поэтому слежу, чтобы белки, жиры и углеводы были в пределах нормы в моих пирогах. 
Среди моих любимых рецептов – пирог с яблоками и ореховый пирог с добавлением шотландского виски. 
У меня есть пара советов для тех, кто хочет печь пироги. 
Пшеничная мука должна быть просеяна, а духовой шкаф должен быть разогрет заранее. 
Еще я люблю красиво подавать пироги своим родным и близким. 
Для этого я использую три красивые чашки и тарелки с росписью, которые мне подарила моя сестра. 
Я завариваю чай, заранее купленный на рынке, и приглашаю всех на чаепитие с пирогом. 
Это наша семейная традиция, моя сестра с детства пила чай с пирогом."""

Правильные части речи:

In [260]:
right_list = ['PRON', 'ADJF', 'NOUN', 'VERB', 'NOUN', 'PRON', 'VERB', 'ADVB', 
              'VERB', 'ADVB', 'VERB', 'CONJ', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 
              'VERB', 'PREP', 'NOUN', 'NOUN', 'PREP', 'PRON', 'NOUN', 'PREP', 
              'PRON', 'ADJF', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'ADJF', 
              'NOUN', 'PREP', 'NOUN', 'ADJF', 'NOUN', 'PREP', 'PRON', 'VERB', 
              'NOUN', 'NOUN', 'PREP', 'PRON', 'PRON', 'VERB', 'VERB', 'NOUN', 
              'ADJF', 'NOUN', 'ADJF', 'VERB', 'VERB', 'CONJ', 'ADJF', 'NOUN', 
              'ADJF', 'VERB', 'VERB', 'ADVB', 'PRCL', 'PRON', 'VERB', 'ADVB', 
              'VERB', 'NOUN', 'PRON', 'NOUN', 'CONJ', 'NOUN', 'PREP', 'PRON', 
              'PRON', 'VERB', 'NUM', 'ADJF', 'NOUN', 'CONJ', 'NOUN', 'PREP', 
              'NOUN', 'PRON', 'PRON', 'VERB', 'PRON', 'NOUN', 'PRON', 'VERB', 
              'NOUN', 'ADVB', 'VERB', 'PREP', 'NOUN', 'CONJ', 'VERB', 'PRON', 
              'PREP', 'NOUN', 'PREP', 'NOUN', 'PRON', 'PRON', 'ADJF', 'NOUN',
              'PRON', 'NOUN', 'PREP', 'NOUN', 'VERB', 'NOUN', 'PREP', 'NOUN']

В английском тексте предполагается, что будут сложными случаи конверсии: studies/study, reading, finding, research, а также многозначные слова (well)

In [261]:
english_text = """Sara Black is the Oxford University student. 
She studies philology because she loves books more than everything else in the world.
She spends all her free time reading books. 
When she was a child, she wrote a tale about a princess and a magic well. 
Her main area of research is myths of Ancient Greece and Rome Empire about finding a treasure. 
Her father told her these stories when they were in Greece many years ago.
When she decided to study philology, she chose this topic for her first research. 
Now she is studying Ancient Greek and Latin to work with original texts in the future."""

Правильные ответы:

In [262]:
right_e_list = ['NOUN', 'NOUN', 'VERB', 'DET', 'NOUN', 'NOUN', 'NOUN',
                'PRON', 'VERB', 'NOUN', 'CONJ', 'PRON', 'VERB', 'NOUN', 'ADVB', 
                'CONJ', 'NOUN', 'ADVB', 'PREP', 'DET', 'NOUN', 'PRON', 'VERB', 'PRON', 'PRON',
                'ADJF', 'NOUN', 'VERB', 'NOUN', 'CONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'PRON',
                'VERB', 'DET', 'NOUN', 'PREP', 'DET', 'NOUN', 'CONJ', 'DET',
                'NOUN', 'NOUN', 'PRON', 'ADJF', 'NOUN', 'PREP', 'NOUN', 'VERB',
                'NOUN', 'PREP', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'PREP', 
                'VERB', 'DET', 'NOUN', 'PRON', 'NOUN', 'VERB', 'PRON', 'DET',
                'NOUN', 'CONJ', 'PRON', 'VERB', 'PREP', 'NOUN', 'ADJF', 'NOUN',
                'ADVB', 'CONJ', 'PRON', 'VERB', 'CONJ', 'VERB', 'NOUN', 'PRON',
                'VERB', 'DET', 'NOUN', 'PREP', 'PRON', 'NUM', 'NOUN', 'ADVB', 
                'PRON', 'VERB', 'VERB', 'ADJF', 'NOUN', 'CONJ', 'NOUN', 'CONJ',
                'VERB', 'PREP', 'ADJF', 'NOUN', 'PREP', 'DET', 'NOUN']

## Анализ текста на русском языке.

Функция, чтобы привести теги к тем, что используются в правильных ответах:

In [263]:
def to_right(l):
  new_list = []
  russian_dict = {'NPRO':'PRON', 'ADJS':'ADJF', 'INFN':'VERB','PRTS':'VERB', 
                  'PRTF':'VERB', 'A':'ADJF', 'ADV':'ADVB', 'ADVPRO':'PRON', 
                  'ANUM':'NUM', 'APRO':'PRON', 'COM':'NOUN', 
                  'PART':'PRCL', 'PR':'PREP', 'S':'NOUN', 'SPRO':'PRON', 
                  'V':'VERB', 'DET':'PRON', 'ADJ':'ADJF', 'ADV':'ADVB', 
                  'SCONJ':'CONJ', 'AUX':'VERB', 'CCONJ':'CONJ',
                  'ADP':'PREP', 'PUNCT':''}
  for i in l:
    if i in russian_dict and russian_dict[i] !='':
      new_list.append(russian_dict[i])
    elif i not in russian_dict:
      new_list.append(i)
  return new_list

### Анализ с помощью PyMorphy:

In [264]:
token_text = wpt.tokenize(russian_text)
pymorphy_list = []
for word in token_text: 
    analyze = morph_parse.parse(word)
    pos = analyze[0].tag.POS
    if pos != None:
        pymorphy_list.append(pos)
pymorphy_list = to_right(pymorphy_list)

Посчитаем долю верных ответов для PyMorphy:

In [265]:
accuracy_score(right_list, pymorphy_list)

0.8392857142857143

### Анализ с помощью MyStem:

In [266]:
text = m.analyze(russian_text)

In [267]:
mystem_list = []
for word in text:
  if 'analysis' in word:
    parse = word.get('analysis')[0]
    pos = parse['gr']
    pos = pos.split('=')[0]
    pos = pos.split(',')[0]
    mystem_list.append(pos)

Посчитаем долю верных ответов:

In [268]:
mystem = to_right(mystem_list)
accuracy_score(right_list, mystem)

0.9553571428571429

### Анализ с помощью Natasha:

In [269]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

In [270]:
doc = Doc(russian_text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
natasha_list = [i.pos for i in doc.tokens]

Посчитаем долю верных ответов для Natasha:

In [271]:
natasha = to_right(natasha_list)
accuracy_score(right_list, natasha)

0.9642857142857143

Как мы видим, лучше всего разметил POS-тэггер __Natasha__.


## Анализ текста на английском языке

Функция, которая приводит ответы POS-тэггеров к тэгам из правильных ответов: 

In [272]:
def eng_to_right(l):
  eng_dict = {'ADJ':'ADJF', 'ADP':'PREP', 'ADV':'ADVB', 'AUX':'VERB', 
                'CCONJ':'CONJ', 'PROPN':'NOUN', 'SCONJ':'CONJ', 'CC':'CONJ', 
                'CD':'NUM', 'DT':'DET', 'EX':'PRON', 'IN':'PREP', 'JJ':'ADJF', 
                'JJR':'ADJF', 'JJS':'ADJF', 'MD':'VERB', 'NN':'NOUN', 
                'NNP':'NOUN', 'NNS':'NOUN', 'PDT':'PRON', 'PRP':'PRON', 
                'PRP$':'PRON', 'RB':'ADVB', 'RBR':'ADVB', 'RBS':'ADVB', 
                'RP':'PART', 'TO':'CONJ', 'VB':'VERB', 'VBD':'VERB', 
                'VBG':'VERB', 'VBN':'VERB', 'VBP':'VERB', 'VBZ':'VERB', 
                'WDT':'PRON', 'WP':'PRON', 'WRB':'PRON'}
  new_list = []
  for i in l:
    if i in eng_dict:
      new_list.append(eng_dict[i])
    elif i != 'PUNCT' and i != 'SPACE' and i.isalpha():
      new_list.append(i)
  return new_list

### Анализ с помощью NLTK

In [273]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [274]:
text = wpt.tokenize(english_text)
nltk_list = nltk.pos_tag(text)
nltk_list = [i[1] for i in nltk_list]

Посчитаем долю верных ответов для NLTK:

In [275]:
nltk_l = eng_to_right(nltk_list)
accuracy_score(right_e_list, nltk_l)

0.8867924528301887

### Анализ с помощью Flair

In [276]:
sentence = Sentence(english_text)
tagger = SequenceTagger.load('pos')
tagger.predict(sentence)

2020-10-16 19:17:17,337 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt


In [277]:
flair_str = sentence.to_tagged_string()

Из строчки получим список:

In [278]:
flair_l = re.findall(r'(?<=\<)[A-Z]+\$?(?=\>)', flair_str)
flair_list = eng_to_right(flair_l)

Посчитаем долю верных ответов:

In [279]:
accuracy_score(right_e_list, flair_list)

0.9245283018867925

### Анализ с помощью SpaCy

In [280]:
nlp = spacy.load("en_core_web_sm")

doc = nlp(english_text)

spacy_list = []
for i, s in enumerate(doc.sents):
    for t in s:
        spacy_list.append(t.pos_)

Посчитаем долю верных ответов:

In [281]:
spacy_list = eng_to_right(spacy_list)
accuracy_score(right_e_list, spacy_list)

0.8679245283018868

## Функция, создающая группа

Возьмем три группы: "не" + прилагательное, прилагательное + существительное и наречие + глагол, так как эти группы больше всего скажут нам об _оценке_ пользователя, так как просто слово "хороший" нам даст неправильную оценку, если перед ним идет отрицание. Так же полезно знать, что именно характеризуют оценочные прилагательные и наречия.

Функция написана с помощью Natasha:

In [282]:
def finding_groups(text):
  groups_list = []
  pos_tags = []
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for i in doc.tokens:
    i.lemmatize(morph_vocab)
  pos_tags = [(i.lemma, i.pos) for i in doc.tokens]
  n_gram_list = []
  for i in range(len(pos_tags)):
    lemma = pos_tags[i][0]
    tag = pos_tags[i][1]
    if (i + 1) != len(pos_tags):
      next_tag = pos_tags[i+1][1]
      next_lemma = pos_tags[i+1][0]
      if lemma == 'не' and next_tag == 'ADJ':
        n_gram_list.append(' '.join([lemma, next_lemma]))
      elif tag == 'ADJ' and next_tag == 'NOUN':
        n_gram_list.append(' '.join([lemma, next_lemma]))
      elif tag == 'ADV' and next_tag == 'VERB':
        n_gram_list.append(' '.join([lemma, next_lemma]))
  return n_gram_list

Посмотрим, какие группы функция нашла для нашего текста на русском:

In [283]:
finding_groups(russian_text)

['любимый хобби',
 'правильно питаться',
 'поэтому следить',
 'любимый рецепт',
 'шотландский виски',
 'пшеничный мука',
 'духовой шкаф',
 'красиво подавать',
 'красивый чашка',
 'заранее купить',
 'семейный традиция']